<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Discrete-Laplacian" data-toc-modified-id="Discrete-Laplacian-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Discrete Laplacian</a></span><ul class="toc-item"><li><span><a href="#Performances-Eval" data-toc-modified-id="Performances-Eval-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Performances Eval</a></span><ul class="toc-item"><li><span><a href="#Profiling" data-toc-modified-id="Profiling-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Profiling</a></span></li></ul></li></ul></li><li><span><a href="#Visualization" data-toc-modified-id="Visualization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Visualization</a></span><ul class="toc-item"><li><span><a href="#Interactive-2D" data-toc-modified-id="Interactive-2D-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Interactive 2D</a></span></li><li><span><a href="#Slicing-3D" data-toc-modified-id="Slicing-3D-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Slicing 3D</a></span></li><li><span><a href="#Plotly-Volume" data-toc-modified-id="Plotly-Volume-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Plotly Volume</a></span></li><li><span><a href="#Voxel-Plot" data-toc-modified-id="Voxel-Plot-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Voxel Plot</a></span></li></ul></li><li><span><a href="#Generate-Video" data-toc-modified-id="Generate-Video-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Generate Video</a></span><ul class="toc-item"><li><span><a href="#3D" data-toc-modified-id="3D-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>3D</a></span></li><li><span><a href="#Rerun-old-configs" data-toc-modified-id="Rerun-old-configs-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Rerun old configs</a></span></li></ul></li><li><span><a href="#Parameters-Grid-Search" data-toc-modified-id="Parameters-Grid-Search-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Parameters Grid Search</a></span></li></ul></div>

# Intro
This notebook explores introductory concepts and interactive examples of **reaction-diffusion systems**, which model the evolution of one or more variables subjects to two processes:
* reaction: transformation from one state to another
* diffusion: expansion across space

[Reaction-Diffusion Tutorial by Karl Sims](http://karlsims.com/rd.html)

$$ A^{\prime } = A + (D_A \nabla^2 A - AB^2 + \text{f} (1-A)) \Delta t $$
$$ B^{\prime } = B + (D_B \nabla^2 B + AB^2 - (k+\text{f})) \Delta t $$

See also the *Dynamical Systems* notebook.

In [1]:
# Basic libraries import
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
from matplotlib import animation
from pathlib import Path
from datetime import datetime
import cv2
from tqdm import tqdm

# Plotting
%matplotlib notebook

sns.set_context("paper")
sns.set_style("darkgrid")

# Local utils
from ReactionDiffusionSystem import ReactionDiffusionSystem, get_init_state, get_polygon_mask, get_cube_mask, get_sphere_mask
from ReactionDiffusionSystem import ReactionDiffusionException
from ReactionDiffusionSystem import SYSTEM_CORAL_CONFIG, SYSTEM_BACTERIA_CONFIG, SYSTEM_SPIRALS_CONFIG, SYSTEM_ZEBRA_CONFIG
plt.rcParams['animation.ffmpeg_path'] = str(Path.home() / "anaconda3/envs/image-processing/bin/ffmpeg")

from ds_utils.video_utils import generate_video

%load_ext autoreload
%autoreload 2

# Discrete Laplacian
The Laplace operator has an analog discrete version for discrete grids.

In two dimensions can be approximated via "five-point stencil finite-difference method". 

In [ ]:
from ReactionDiffusionSystem import discrete_laplacian, discrete_laplacian_convolve, kernel_2d, kernel_2d_2

In [ ]:
def discrete_laplacian_fivepoint(Z, dx):
    Ztop = Z[0:-2, 1:-1]
    Zleft = Z[1:-1, 0:-2]
    Zbottom = Z[2:, 1:-1]
    Zright = Z[1:-1, 2:]
    Zcenter = Z[1:-1, 1:-1]
    return (Ztop + Zleft + Zbottom + Zright -
            4 * Zcenter) / dx**2

In [ ]:
test_Z = np.ones((4,4)) * 2.
test_Z[1,1] = 0
test_Z

In [ ]:
discrete_laplacian_fivepoint(test_Z, dx=1)

In [ ]:
# use numpy roll in the target directions
discrete_laplacian(test_Z, None)

In [ ]:
# use scipy convolve2d
discrete_laplacian_convolve(test_Z, kernel_2d_2)

In [ ]:
# use cv2 filter
cv2.filter2D(test_Z, -1, kernel_2d, borderType=cv2.BORDER_REFLECT)

In [ ]:
rand_grid = np.random.randint(0, 10, (200,200))
%timeit discrete_laplacian(rand_grid)
%timeit discrete_laplacian_convolve(rand_grid)
%timeit cv2.filter2D(rand_grid*1.0, -1, kernel_2d, borderType=cv2.BORDER_REFLECT)

## Performances Eval

In [ ]:
def system_run(size=100, steps=10):
    system_shape = tuple([size]*3)
    rf_system = ReactionDiffusionSystem(system_shape, config,
                                       lambda shape: get_init_state(shape, 0.2, None, 0.5, .2),
                                        validate_change_threshold=-0.001)
    rf_system.run_simulation(steps)

In [ ]:
%timeit system_run()

### Profiling

In [ ]:
%%prun -s cumulative -l 30 -r
# We profile the cell, sort the report by "cumulative
# time", limit it to 30 lines

system_run(size=100, steps=50)

# Visualization 

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
from ipywidgets import Button

%matplotlib notebook 
from matplotlib import animation

## Interactive 2D

In [ ]:
def plot_animation(fig, ax, rf_system, nb_frames, simulation_steps, interval=100):
    im = ax.imshow(rf_system.B, cmap=plt.cm.Blues, interpolation='bilinear', extent=[-1, 1, -1, 1])

    def animate(i, rf_system, simulation_steps):
        rf_system.run_simulation(simulation_steps)
        im.set_data(rf_system.B)

    # Animate
    ani = animation.FuncAnimation(fig, animate, frames=nb_frames, interval=interval, 
                                  fargs=[rf_system, simulation_steps])
    return ani

In [ ]:
# setup plot
nb_frames = 200
simulation_steps = 30

img_width = img_height = 100
fig, ax = plt.subplots(dpi=100, figsize=(5, 5))
plt.axis('off')

# system config
config = SYSTEM_SPIRALS_CONFIG.copy()
config['COEFF_A'] = .21
config['COEFF_B'] = 0.05
#config['FEED_RATE'] = 0.0625
#config['KILL_RATE'] = 0.05

# system init
system_shape = (img_width, img_height)
mask = get_polygon_mask(system_shape, 4, system_shape[0]//10, np.array(system_shape) // 2)
mask = cv2.resize(cv2.imread("C:/Users/User/Downloads/rd_mask.png", cv2.IMREAD_GRAYSCALE), system_shape)
rf_system = ReactionDiffusionSystem(system_shape, config,
                                   lambda shape: get_init_state(shape, 0.0, mask),
                                   validate_change_threshold=0.0001)

# plot
plot_animation(fig, ax, rf_system,
                nb_frames=nb_frames, simulation_steps=simulation_steps, interval=10)

In [ ]:
# setup plot
nb_frames = 100
simulation_steps = 10

img_width = img_height = 100
fig, ax = plt.subplots(dpi=100, figsize=(5, 5))
plt.axis('off')

@interact
def i_style_mixing(coeff_a = np.linspace(0.10, 0.2, 10), coeff_b = np.linspace(0.01, 0.1, 10), 
                   feed_rate = np.linspace(0.03, 0.05, 10), kill_rate = np.linspace(0.05, 0.08, 10)):
    config = {'COEFF_A': coeff_a, 'COEFF_B': coeff_b, 
              'FEED_RATE': feed_rate, 'KILL_RATE': kill_rate}
    return plot_animation(fig, ax, (img_width, img_height), config,
                  nb_frames=nb_frames, simulation_steps=simulation_steps, interval=10)

## Slicing 3D

In [ ]:
system_shape = tuple([50]*3)
mask = get_cube_mask(system_shape, system_shape[0]//10, np.array(system_shape) // 2)
rf_system = ReactionDiffusionSystem(system_shape, SYSTEM_CORAL_CONFIG,
                                   lambda shape: get_init_state(shape, 0.0, mask=mask))

rf_system.run_simulation(1000)

In [ ]:
%matplotlib notebook
from matplotlib.widgets import Slider
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
plt.axis('off')
idx0 = 3
l = ax.imshow(rf_system.get_concentration()[idx0], cmap=plt.cm.copper,
          interpolation='bilinear',
          extent=[-1, 1, -1, 1])

axidx = plt.axes([0.25, 0.15, 0.65, 0.03])
slidx = Slider(axidx, 'index', 0, system_shape[2], valinit=idx0, valfmt='%d')

def update(val):
    idx = slidx.val
    l.set_data(rf_system.get_concentration()[int(idx)])
    fig.canvas.draw_idle()
slidx.on_changed(update)

plt.show()

## Plotly Volume

In [ ]:
import plotly.graph_objects as go

In [ ]:
side = 50
system_shape = tuple([side]*3)

rf_system = ReactionDiffusionSystem(system_shape, SYSTEM_ZEBRA_CONFIG,
                                   lambda shape: get_init_state(shape, 'CENTER'))
isomin=0.06
isomax=0.5

# store the state of multiple simulation to replay in the 3D plot
res = []
for i in range(10):
    rf_system.run_simulation(300)
    res.append(rf_system.B.copy())

In [ ]:
def volume_data(X, Y, Z, values, isomin, isomax):
    return go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=values.flatten(),
        isomin=isomin,
        isomax=isomax,
        opacity=0.7, # needs to be small to see through all surfaces
        surface_count=10, # needs to be a large number for good volume rendering
    )

In [ ]:
X, Y, Z = np.mgrid[0:side, 0:side, 0:side]

fig = go.Figure(
    data=[volume_data(X, Y, Z, res[0], isomin, isomax)],
    layout=go.Layout(
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=[go.Frame(data=volume_data(X, Y, Z, v, isomin, isomax)) for v in res]
    )
fig.show()

## Voxel Plot

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2],
                linewidth=0.2, antialiased=True)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#xx, yy, zz = np.where(rf_system.B> 0.003)
ax.voxels(rf_system.B> 0.01)

# Generate Video

In [ ]:
def draw(U):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(U, cmap=plt.cm.copper,
              interpolation='bilinear',
              extent=[-1, 1, -1, 1])
    ax.set_axis_off()

In [ ]:
rf_system = ReactionDiffusionSystem((100, 100), SYSTEM_BACTERIA_CONFIG)

In [ ]:
rf_system.run_simulation(1000, delta_t=1.2)

In [ ]:
draw(rf_system.B)

In [ ]:
def base_frame_gen(frame_count, rf_system, simulation_steps):
    rf_system.run_simulation(simulation_steps)
    img = cv2.normalize(rf_system.get_concentration(), None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return img

In [ ]:
def touch_frame_gen(frame_count, rf_system, simulation_steps):
    rf_system.run_simulation(simulation_steps)
    #if i == nb_frames//2:
    #    center = np.array(rf_system.shape) // 2
    #    r = np.array(rf_system.shape) // 10
    #    rf_system.B[center[0] - r[0]:center[0] + r[0], center[1] - r[1]:center[1] + r[1]] = 0.25
    img = cv2.normalize(rf_system.B, None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return img

In [ ]:
generate_video(str(out_path/"tmp.mp4"), (rf_system.shape[1], rf_system.shape[0]),
               frame_gen_fun = lambda i: base_frame_gen(i, rf_system, 20),
               nb_frames = 10)

## 3D

In [ ]:
def frame_gen_3d(frame_count, z_coord, rf_snapshots):
    img = cv2.normalize(rf_snapshots[frame_count][z_coord], None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return img

In [ ]:
config = {'COEFF_A': 0.12, 'COEFF_B': 0.07, 'FEED_RATE': 0.041249999999999995, 'KILL_RATE': 0.058, 'steps': 30, 'random_influence': 0.0, 'validate_change_threshold': 1e-06, 'nb_frames': 228}
config = SYSTEM_SPIRALS_CONFIG

# create 3d system
system_shape = tuple([50]*3)
mask = get_cube_mask(system_shape, system_shape[0]//5, np.array(system_shape) // 2)
rf_system = ReactionDiffusionSystem(system_shape, config,
                                   lambda shape: get_init_state(shape, random_influence=0.0, mask=mask),
                                   validate_change_threshold=1.e-6)

In [ ]:
rf_snapshots = []
nb_frames = 240
nb_steps = 30
for i in range(nb_frames):
    rf_system.run_simulation(nb_steps)
    rf_snapshots.append(rf_system.B)
    #if i%50 == 0:
    #    print(i)

In [ ]:
out_path = Path.home() / 'reaction_diffusion'
out_path.mkdir(exist_ok=False, parents=True)
#for z_coord in range(system_shape[2]):
#    img = cv2.normalize(rf_system.B[z_coord], None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
#    cv2.imwrite(str(out_path / f'{z_coord}.png'), img)

for z_coord in range(system_shape[0]):
    generate_video(str(out_path/f"{z_coord}.mp4"), 
                   (rf_system.shape[2], rf_system.shape[1]),
                   frame_gen_fun = lambda i: frame_gen_3d(i, z_coord, rf_snapshots),
                   nb_frames = nb_frames, is_color=False)

## Rerun old configs

In [ ]:
from ast import literal_eval
target_path = Path.home() / 'reaction_diffusion/3d_slicing/100x100'
out_path = Path.home() / 'reaction_diffusion/3d_slicing/100x100_slow'
out_path.mkdir(exist_ok=True, parents=True)
NUM_FRAMES = 240
with open(str(target_path / 'logs.txt'), 'r') as f:
    for run, line in enumerate(f):
        print(f'#####################')
        print(f'Run {run}')
        rf_snapshots = []
        config = literal_eval(line)
        config['steps'] = 10
        if config['nb_frames'] < 100:
            continue

        # init reaction diffusion system
        system_shape = tuple([100]*3)
        mask = get_cube_mask(system_shape, system_shape[0]//5, np.array(system_shape) // 2)
        system_init_fun = lambda shape: get_init_state(shape, random_influence=config['random_influence'], mask=mask)
        rf_system = ReactionDiffusionSystem(system_shape, config, system_init_fun,
                                            validate_change_threshold=config['validate_change_threshold'])

        # run and store snapshot
        for i in range(NUM_FRAMES):
            try:
                rf_system.run_simulation(config['steps'])
                rf_snapshots.append(rf_system.B)
            except ReactionDiffusionException as e:
                print(f'System throw exception at frame {i} {e}')
                break
            if i % 50 == 0:
                print('Frame ', i)

        # write out numpy 4D tensor
        np.save(out_path / f'run_{run}.npy', np.array(rf_snapshots, dtype=np.float16))

        # write out as sliced videos
        run_out_path = out_path / f'vid_run_{run:03}'
        run_out_path.mkdir(exist_ok=True, parents=True)
        for z_coord in range(system_shape[0]):
            generate_video(str(run_out_path / f"{z_coord}.mp4"),
                           (rf_system.shape[2], rf_system.shape[1]),
                           frame_gen_fun=lambda i: frame_gen_3d(i, z_coord, rf_snapshots),
                           nb_frames=len(rf_snapshots), is_color=False, disable_tqdm=True)

# Parameters Grid Search 

In [2]:
from ds_utils.sim_utils import named_configs
from ds_utils.point_cloud_utils import prepare_for_ply, write_to_ply

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
out_path = Path.home() / 'Documents/graphics/generative_art_output/rd'

In [5]:
import cupy as cp
def normalize_snapshots(snapshots):
    norm_s = cp.asnumpy(cp.array(snapshots))
    norm_s = (norm_s/norm_s.max())*255
    return norm_s.astype(np.uint8)

In [6]:
def get_mean_edges(norm_snapshots, edge_size):
    img = norm_snapshots[-1][edge_size:-edge_size, edge_size:-edge_size, edge_size:-edge_size]
    # remove noise to improve edge detection results
    blurred_img = cv2.GaussianBlur(img, (5, 5), 0)
    # canny edge detection
    edges = cv2.Canny((blurred_img * 255).astype('uint8'), 100, 200)
    return edges.mean()

In [7]:
nb_vals = 3
grid_search_params = {
    'COEFF_A': np.linspace(0.12, 0.16, nb_vals),
    'COEFF_B': np.linspace(0.06, 0.08, nb_vals),
    'FEED_RATE': np.linspace(0.02, 0.0625, nb_vals),
    'KILL_RATE': np.linspace(0.05, 0.066, nb_vals),
}
configs = list(named_configs(grid_search_params))

In [8]:
system_size = 100
system_shape = tuple([system_size]*3)
render_dir = out_path / '3d_ply' / f'circle_shifted_{system_size}_size'
render_dir.mkdir(exist_ok=False, parents=True)
nb_frames = 180
min_frames = int(0.8 * nb_frames)

mean_edges_threshold = 30
generate_ply = True
ply_threshold = 70

config = {}
config['simulation_steps'] = 20
config['random_influence'] = 0.
config['boundary'] = 'reflect'
config['validate_change_threshold'] = 1.e-6

# mask
mask_size_ratio = 1/2.5
# mask = get_cube_mask(system_shape, system_shape[0]*mask_size_ratio, 
#                      center=np.array(system_shape) // 1)
mask = get_sphere_mask(system_shape, [system_shape[0]*mask_size_ratio, (system_shape[0]*mask_size_ratio)+20], 
                     center=np.array(system_shape) // 1)

system_init_fun = lambda shape: get_init_state(shape, random_influence=config['random_influence'], mask=mask)

nb_runs = 1
#hexa_paths = list((Path("automaton_hexagonal/flat_hexa_logo").glob("*/*.png"))
for run_idx in range(nb_runs):
    #seed_image = cv2.resize(cv2.imread(str(hexa_paths[np.random.randint(len(hexa_paths))])) / 255, system_shape[:2])
    with open(str(render_dir / "logs.txt"), 'w+') as f:
        for config_idx, param_config in tqdm.tqdm_notebook(enumerate(configs)):
            print(f'#####################')
            print(f'Run {run_idx} - config {config_idx}')
            run_dir = render_dir / 'run_{}_config_{}'.format(run_idx, config_idx)
            
            rf_snapshots = []
            config.update(param_config._asdict())
            rf_system = ReactionDiffusionSystem(system_shape, config, system_init_fun,
                                               validate_change_threshold=config['validate_change_threshold'])

            #if seed_image is not None:
            #    rf_system.B[np.where(seed_image[:, :, 1]>0.1)] =  0.25
            #    rf_system.A[np.where(seed_image[:, :, 1]>0.1)] =  0.50
            
            # run and store snapshot
            for i in range(nb_frames):
                try:
                    rf_system.run_simulation(config['simulation_steps'])
                    rf_snapshots.append(rf_system.B)
                except Exception as e:
                    print(f'System throw exception at frame {i} {e}')
                    break
                if i % 50 == 0:
                    print('Frame ', i)

            # write out config
            config['nb_frames'] = len(rf_snapshots)
            f.write(str(config) + '\n')
            
            # discard if too short
            if len(rf_snapshots) < min_frames:
                continue
                
            norm_snapshots = normalize_snapshots(rf_snapshots)
            
            # discard if last frame not rich enough
            mean_edges = get_mean_edges(norm_snapshots,  edge_size=int(system_size//5))
            if mean_edges < mean_edges_threshold:
                print(f'Skipping because low mean_edges ({mean_edges:0.2f})')
                continue
            else:
                print(f'mean_edges: {mean_edges:0.2f}')
                
            run_dir.mkdir(exist_ok=False)
                
            # save each frame to ply
            if generate_ply:
                print('Writing ply files')
                out_ply = run_dir / 'ply'
                out_ply.mkdir(exist_ok=False)
                ply_snapshots = prepare_for_ply(norm_snapshots, ply_threshold)
                for frame in np.arange(norm_snapshots.shape[0]):
                    tmp = ply_snapshots[frame]
                    tmp = tmp[tmp[:,-1] >= ply_threshold]
                    write_to_ply(tmp, out_ply / f'frame_{frame:03d}.ply')
            
            # generate video (single slice only)
            print('Generating video')
            out_video = run_dir / 'run.mp4'
            generate_video(str(out_video), (system_shape[1], system_shape[0]),
                           frame_gen_fun=lambda i: norm_snapshots[i][(system_shape[-1]//1)-1],
                           nb_frames=len(rf_snapshots), is_color=False, disable_tqdm=True)

0it [00:00, ?it/s]

#####################
Run 0 - config 0
Frame  0
Frame  50
Frame  100
Frame  150
mean_edges: 46.54
Writing ply files


1it [01:32, 92.79s/it]

Generating video
#####################
Run 0 - config 1
Frame  0


2it [01:33, 65.13s/it]

...

Skipping because low mean_edges (0.00)
#####################
Run 0 - config 79
Frame  0
Frame  50
Frame  100
Frame  150


80it [38:19, 36.41s/it]

Skipping because low mean_edges (0.00)
#####################
Run 0 - config 80
Frame  0


81it [38:19, 28.39s/it]

System throw exception at frame 7 abs_change = 0.0
